### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [1]:
import pandas as pd
import numpy as np
import pickle

import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_touchdowns, stats_for_trailing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore',
                     category = SettingWithCopyWarning)

### Import Data

Let's import the dataframe that we will be using for modelling

In [3]:
data = pd.read_csv('Data/weekly_data.csv')

data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [4]:
data.isna().sum()

Unnamed: 0      0
Unnamed: 0.1    0
GameKey         0
PlayerID        0
Season          0
               ..
HomePoints      0
HomeYards       0
AwayTeam        0
AwayPoints      0
AwayYards       0
Length: 65, dtype: int64

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [5]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df.loc[:,column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    df_out.sort_index(inplace = True)
    return df_out
    

In [6]:
stats_for_trailing = ['FantasyPointsPPR']

In [7]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data[data['Name'] == 'Tom Brady'][['FantasyPointsPPR', 'TAFantasyPointsPPR']].head(35)

,FantasyPointsPPR,TAFantasyPointsPPR
14,21.28,18.332
506,16.36,18.332
1077,7.52,18.332
1454,18.96,18.332
1860,27.54,18.332
2325,21.80,18.332
2758,21.68,18.436
3212,13.76,19.500
3665,15.66,20.748
4128,11.76,20.088


### Train Test Split

In [8]:
# Separate data from the target
# y = data['FantasyPointsPPR']

# Apply a log transform to the target to see how it impacts prediction
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [9]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.2,
                                                    shuffle = False,
                                                   random_state = 13)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [10]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats.append('TAFantasyPointsPPR')

In [11]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer)])

<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [12]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                       'Name',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']

numerical_columns = trailing_stats


In [13]:
# Create a custom function to generate a log-transformed version of continuous data with a constant 5 added prior to the transform.

def LogShift(X):
    
    X_10 = X + 10
    X_log = np.log(X_10)
    
    return X_log

LogShiftTransformer = FunctionTransformer(LogShift)

In [14]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder(handle_unknown = 'ignore'))])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'mean')),
                               ('log', LogShiftTransformer)])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [15]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', LinearRegression())])

In [16]:
# Set param grid values, parameters for grid search 

param_grid = {}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                           scoring = 'neg_mean_squared_error',
                          cv = 5,
                          verbose = 3)

In [17]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('engineering',
                                        Pipeline(steps=[('touchdown',
                                                         FunctionTransformer(func=<function get_touchdowns at 0x7f9db00419d0>)),
                                                        ('trailing',
                                                         FunctionTransformer(func=<function trailing_stats_mean at 0x7f9db0041820>)),
                                                        ('tier',
                                                         FunctionTransformer(func=<function get_tiers at 0x7f9db0041940>))])),
                                       ('prep',
                                        ColumnTransformer(transform...
                                                                          'TAFumbles',
                                                                          'TAFumblesLost',
                                   

In [18]:
grid_search.best_score_

-33.0293777457036

In [19]:
grid_search.best_params_

{}

In [20]:
preds = grid_search.predict(X_test)

In [21]:
mean_squared_error(y_test, preds)

34.395925594054674

In [22]:
mean_absolute_error(y_test, preds)


4.217621264587435

In [23]:
r2_score(y_test, preds)

0.41117999867838695

Let's dig into the predictions a little bit

In [31]:
df_preds = pd.DataFrame(y_test)

In [32]:
preds_df = pd.DataFrame(y_test)
preds_df['Prediction'] = preds 
preds_df['Actual Value'] = np.exp(preds_df['FantasyPointsPPR']) - 5
preds_df['Prediction Value'] = np.exp(preds_df['Prediction']) - 5
preds_df.head(50)

,FantasyPointsPPR,Prediction,Actual Value,Prediction Value
25230,0.0,0.850793,-4.0,-2.658498
25231,0.0,-4.316983,-4.0,-4.986660
25232,0.0,3.558677,-4.0,30.116721
25233,0.0,-3.496689,-4.0,-4.969702
25234,0.0,5.818900,-4.0,331.601713
25235,0.0,1.997826,-4.0,2.373011
25236,0.0,1.284281,-4.0,-1.387929
25237,0.0,2.387221,-4.0,5.883204
25238,0.0,4.024857,-4.0,50.972309
25239,0.0,6.470269,-4.0,640.657447


### Save model


In [26]:
import pickle

pickle.dump(grid_search, open('Pickles/log_transform_linear.pickle', 'wb'))